In [33]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest,chi2


In [34]:
df=pd.read_csv('/content/covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [35]:
x_tr,x_te,y_tr,y_te=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2,random_state=42)

In [36]:
x_tr.isnull().sum()

,0
age,0
gender,0
fever,9
cough,0
city,0


In [37]:
trf1 = ColumnTransformer([
    ('fever_imputer', SimpleImputer(), [2])
], remainder='passthrough')

In [38]:

trf2= ColumnTransformer([
    ('cough_ordinal',OrdinalEncoder(categories=[['Mild','Strong']]),[3])
],remainder='passthrough')

In [39]:
trf3 = ColumnTransformer([
    ('gender_city',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1]),
  ('city_onehot',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[4]),
], remainder='passthrough')

In [40]:
trf4=ColumnTransformer([
    ('scaler',MinMaxScaler(),slice(0,9))
])

In [41]:
trf5= DecisionTreeClassifier()

In [42]:
pipe=Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
   ])

In [43]:
pipe.fit(x_tr,y_tr)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('fever_imputer',
                                                  SimpleImputer(), [2])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cough_ordinal',
                                                  OrdinalEncoder(categories=[['Mild',
                                                                              'Strong']]),
                                                  [3])])),
                ('trf3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('gender_city',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1]),
                                                 ('city_onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [4])])),
                ('trf4',
                 ColumnTransformer(transformers=[('scaler', MinMaxScaler(),
                                                  slice(0, 9, None))])),
                ('trf5', DecisionTreeClassifier())])

In [44]:
y_pred= pipe.predict(x_te)

In [46]:
from sklearn.metrics import accuracy_score
accuracy_score(y_te,y_pred)

0.35